In [1]:
# importing libraries for data manipulation and visualization
import pandas as pd
import plotly.express as px
from dash import Dash,dcc,html

In [2]:
# importing Lebron's data for all major stats since 03
lebron_stats = pd.read_csv("lebron_career.csv",parse_dates=["date"])

In [3]:
lebron_stats.head(5)

,game,date,age,team,opp,result,mp,fg,fga,fgp,...,orb,drb,trb,ast,stl,blk,tov,pts,game_score,plus_minus
0,1,2003-10-29,18-303,CLE,SAC,L (-14),42,12,20,0.600,...,2,4,6,9,4,0,2,25,24.7,-9
1,2,2003-10-30,18-304,CLE,PHO,L (-9),41,8,17,0.471,...,2,10,12,8,1,0,7,21,14.7,-3
2,3,2003-11-01,18-306,CLE,POR,L (-19),39,3,12,0.250,...,0,4,4,6,2,0,2,8,5.0,-21
3,4,2003-11-05,18-310,CLE,DEN,L (-4),41,3,11,0.273,...,2,9,11,7,2,3,2,7,11.2,-3
4,5,2003-11-07,18-312,CLE,IND,L (-1),44,8,18,0.444,...,0,5,5,3,0,0,7,23,9.0,-7


In [4]:
lebron_stats.tail(5)

,game,date,age,team,opp,result,mp,fg,fga,fgp,...,orb,drb,trb,ast,stl,blk,tov,pts,game_score,plus_minus
1260,63,2020-08-03,35-217,LAL,UTA,W (+8),35,9,16,0.563,...,1,7,8,9,2,1,4,22,21.0,0
1261,64,2020-08-05,35-219,LAL,OKC,L (-19),30,7,19,0.368,...,1,10,11,4,1,0,0,19,14.4,-8
1262,65,2020-08-08,35-222,LAL,IND,L (-5),35,13,24,0.542,...,1,7,8,7,0,1,1,31,24.0,13
1263,66,2020-08-10,35-224,LAL,DEN,W (+3),38,9,21,0.429,...,0,1,1,12,0,2,5,29,21.4,-10
1264,67,2020-08-13,35-227,LAL,SAC,L (-14),15,6,13,0.462,...,1,2,3,4,0,0,3,17,11.4,-13


In [19]:
lebron_stats["Year"] = lebron_stats.date.dt.year

In [22]:
lbj_yearly = lebron_stats.groupby("Year").agg({"pts":"mean","blk":"mean","ast":"mean","tov":"mean","mp":"mean"
                    ,"fga":"mean"})

In [39]:
px.line(lbj_yearly,x=lbj_yearly.index,y="pts",labels={"pts":"Points Per Game"}
       ,title="Points Per Game").update_layout(
    paper_bgcolor="white",plot_bgcolor="white",title={"x":.5,"y":.85,"font":{"size":30}}
).update_yaxes(
    gridcolor="black"
)

In [40]:
from statsmodels.tsa.arima.model import ARIMA

In [74]:
train_data = lbj_yearly.iloc[
    -12:-5
]  # Use data from 2015 onwards for testing
test_data = lbj_yearly.iloc[-5:]

In [75]:
# Fit the ARIMA model
model = ARIMA(train_data["pts"], order=(5, 1, 1))  #Order AR is comparing previous 5 year averages(2015 & before)
model_fit = model.fit()

# Forecast 5 years ahead
forecast = model_fit.forecast(steps=5)  # Forecasting 5 years
df = forecast.to_frame(name="pts")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning:

No supported index is avail

In [76]:
# Creating yearly forecast for pts for the next year
forecast_dates = pd.Series([2016,2017,2018,2019,2020])
forecast_pts = pd.Series(forecast.array,index=range(0,5))
forecast_df = pd.concat([forecast_pts.rename("pts"),forecast_dates.rename("Date")],axis=1)
forecast_df.set_index("Date",inplace=True)

In [77]:
fig =px.line(lbj_yearly,x=lbj_yearly.index,y="pts",labels={"pts":"Points Per Game"}
       ,title="Points Per Game").update_layout(
    paper_bgcolor="white",plot_bgcolor="white",title={"x":.5,"y":.85,"font":{"size":30}}
).update_yaxes(
    gridcolor="black"
)
fig.add_scatter(x=forecast_df.index,y=forecast_df["pts"],mode="lines")
fig.show()

In [47]:
forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 2016 to 2020
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pts     5 non-null      float64
dtypes: float64(1)
memory usage: 80.0 bytes
